In [1]:
import json
import os
from pprint import pprint

In [2]:
# Cell for all constants

TEST_DIR = ".." + os.sep + 'test' + os.sep
DATA_DIR = os.path.join(os.getcwd(), ".." + os.sep + 'data' + os.sep)
CATS_DIR = os.path.join(DATA_DIR, 'dogs_and_cats')

In [3]:
my_json = \
"""
{
	"layers": {
		"conv2d": {
			"filters": 32,
			"kernel_size": [3, 3],
			"activation": "relu",
			"padding": "same",
			"input_shape": [200, 200, 3]
		},

		"mp2": {
			"pool_size": [2, 2]
		}
	},

	"dataset": {
		"type": "csv",
		"path": "../data/dogs_and_cats"

	},
	"image": {
		"augment": {
			"rotation_range": 40,
			"width_shift_range": 0.2,
			"height_shift_range": 0.2,
			"horizontal_flip": "True",
			"rescale": 0.0039215
		}

	},
	"params": {
		"target_size": [200, 200],
		"batch_size": 64,
		"class_mode": "binary"
	}
}

"""

In [4]:
inputs = json.loads(my_json)

In [6]:
symbol_table = {
    'conv2d' : 'model.add(Conv2D())',
    'mp2' : 'model.add(MaxPooling2D())',
    'dense': 'model.add(Dense())'
}

In [7]:
def get_imports():
    """
    imports the needed modules 
    """
    try:
        return \
        """
# File generated by DLMML Parser
import sys
import os
import numpy
import pandas
from matplotlib import pyplot
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
"""
    except Exception as e:
        print(e)

In [8]:
def init_sequential():
    """
    initializes the model
    """
    try:
        return \
'\nmodel = Sequential()\n'

    except Exception as e:
        print(e)

In [9]:
def image_input():
    """
    req in input json -
    dataset-path -> path to dataset
    image-augment -> params for image augmentation if required
    params -> params for building generator
    """
    base = inputs['dataset']['path']
    test_dir = os.path.join(base, 'test')
    train_dir = os.path.join(base, 'train')
    
    paths = 'base = \'{}\'\n\
train_dir = os.path.join(base, \'train\')\n\
test_dir = os.path.join(base, \'test\')\n'.format(inputs['dataset']['path'])
    i = 'augment = {}\n\
kwargs = {}\n\
train_datagen = ImageDataGenerator(**augment)\n\
test_datagen = ImageDataGenerator(**augment)\n\
train_generator = train_datagen.flow_from_directory(train_dir, **kwargs)\n\
test_generator = test_datagen.flow_from_directory(test_dir, **kwargs)\n'.format(inputs['image']['augment'], inputs['params'])

    return paths+i


In [10]:
def parse(inputs):
    """
    Parser which adds layers (mostly)
    """
    try:
        # init_sequential()
        input_dict = json.loads(inputs)
        input_dict = input_dict['layers']
        generated_code = ''

        #TODO: Take care of indentation and stuff if making functions
        #TODO: Ordering in dict (work around -> ordered_dict)

        for layer in input_dict.keys():
            curr_layer = symbol_table[layer]
            args = str(input_dict[layer])
            curr_layer = curr_layer[:-2] + args[1:-1] + curr_layer[-2:]
            generated_code += curr_layer + '\n'
        print()
        return generated_code
    except Exception as e:
        print(e)


In [11]:
def write_to_file(file_name, input_json):
    try:
        with open(TEST_DIR + file_name, 'w') as f:
            f.write(get_imports())
            f.write(image_input())
            f.write(init_sequential())
            f.write(parse(input_json))
    except:
        print("Exception occured")

In [12]:
write_to_file('test.py', my_json)